In [6]:
import json

def find_opinion_by_lexicon(sens, lexicon_file, rate):
    words_md = ['could', 'may', 'would', 'must', 'might', 'shall', 'ought', 'can']
    find_opinion = []
    score_dic = {}
    with open(lexicon_file) as file_obj:
        pri_list = json.load(file_obj)
    for i, s in enumerate(sens):
        score = 0
        for ss in s:
            if ss.lower() in pri_list or ss in pri_list:
                score += 1
            if ss.lower() in words_md:
                score += 2
        score_dic[i] = score
    
    sorted_dic = sorted(score_dic.items(), key=lambda item:item[1], reverse=True)
    extract_length = int(len(sens)*rate)
    for i, sd in enumerate(sorted_dic):
        if i < extract_length:
            find_opinion.append(sens[sd[0]])
    return find_opinion

In [4]:
# load train set
opinions = []
drivers = []
barriers = []
for i in range(2, 15):
    with open(str(i)+'_batch/opinion_token.json') as file_obj:
        opinion = json.load(file_obj)
        opinions.extend(opinion)
    with open(str(i)+'_batch/driver_token.json') as file_obj:
        driver = json.load(file_obj)
        drivers.extend(driver)
    with open(str(i)+'_batch/barrier_token.json') as file_obj:
        barrier = json.load(file_obj)
        barriers.extend(barrier)
    
with open('1_batch/driver_token.json') as file_obj:
    opinion = json.load(file_obj)
    opinions.extend(opinion) 
    drivers.extend(opinion)
    
with open('1_batch/barrier_token.json') as file_obj:
    opinion = json.load(file_obj)
    opinions.extend(opinion)
    barriers.extend(opinion)

print(len(opinions))
print(len(drivers))
print(len(barriers))

1237
856
381


In [3]:
import import_ipynb
from utils import construct_dict
import os

if 'opinion_dic.json' in os.listdir():
    with open('opinion_dic.json') as file_obj:
        opinion_dic = json.load(file_obj)
else:
    opinion_dic = construct_dict(opinions)
    
if 'lexicon_dic.json' in os.listdir():
    with open('lexicon_dic.json') as file_obj:
        lexicon_dic = json.load(file_obj)
else:
    print('please run unsupervised_lexicon.ipynb first to get lexicon_dic.json')

importing Jupyter notebook from utils.ipynb


In [14]:
import numpy as np
from sklearn import preprocessing

opinion_words = []
opinion_freq = []
lexicon_freq = []
words_md = ['could', 'may', 'would', 'must', 'might', 'shall', 'ought', 'can']
for od in opinion_dic.keys():
    if od in lexicon_dic.keys():
        opinion_words.append(od)
        opinion_freq.append(opinion_dic[od])
        lexicon_freq.append(lexicon_dic[od])
print(len(lexicon_dic.keys()), 'words in lexicon_dic')
print(len(opinion_dic.keys()), 'words in opinion_dic')

opinion_rank = {key: rank for rank, key in enumerate(sorted(opinion_dic.values()), 1)}
lexicon_rank = {key: rank for rank, key in enumerate(sorted(lexicon_dic.values()), 1)}
# print(opinion_rank)
# calculate rank summation
op_sum = 0
lex_sum = 0
for o in opinion_dic:
    freq_n = opinion_dic[o]
    op_sum += opinion_rank[freq_n]
for o in lexicon_dic:
    freq_n = lexicon_dic[o]
    lex_sum += lexicon_rank[freq_n]
    
op_score = {}
lex_score = {}
# record the score 
for o in opinion_dic.keys():
    freq_n = opinion_dic[o]
    op_score[o] = opinion_rank[freq_n]/op_sum
for o in lexicon_dic.keys():
    freq_n = lexicon_dic[o]
    lex_score[o] = lexicon_rank[freq_n]/lex_sum

# sort the need word
need_score = {}
for o in op_score.keys():
    if o in lex_score.keys():
        need_score[o] = op_score[o]-lex_score[o]
sort_rank = sorted(need_score.items(), key=lambda item:item[1], reverse=True)
rank_list_whole = []
for r in sort_rank:
    if r[0] != '[' and r[0] != ']' and r[0] != '’':
        rank_list_whole.append(r[0])

rank_list = rank_list_whole[:150]
json_str = json.dumps(rank_list, indent=4)
with open('rank_150.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)    

NameError: name 'opinion_dic' is not defined

In [6]:
# Sort(Normalization(Opinion_freq) - Normalization(lexicon_freq)) to construct dictionary

opinion_freq = np.array(opinion_freq).reshape(1, -1)
normalizer_opinion = preprocessing.Normalizer().fit_transform(opinion_freq)
normalizer_opinion = list(normalizer_opinion[0])

lexicon_freq = np.array(lexicon_freq).reshape(1, -1)
normalizer_lexicon = preprocessing.Normalizer().fit_transform(lexicon_freq)
normalizer_lexicon = list(normalizer_lexicon[0])

more_value = {}
for i, w in enumerate(opinion_words):
    more_value[w] = normalizer_opinion[i]-normalizer_lexicon[i]
pri_words = sorted(more_value.items(), key=lambda item:item[1], reverse=True)
pri_list_whole = []
i = 0
for p_set in pri_words:
    if p_set[0] not in words_md and opinion_dic[p_set[0]]>1:
        # print(i, p_set, opinion_dic[p_set[0]])
        pri_list_whole.append(p_set[0])
        i += 1
        
# The amount can be adjusted, here choose 100  
pri_list = pri_list_whole[:100]
# json_str = json.dumps(pri_list, indent=4)
# with open('pri_200.json', 'w', encoding='utf-8') as json_file:
    # json_file.write(json_str)
print(pri_list)

['recycling', 'incentive', 'product', 'EoL', 'scheme', 'collection', 'management', 'policy', 'industry', 'design', 'lack', 'economic', 'responsibility', 'regulation', 'waste', 'producer', 'government', 'business', 'recyclers', 'material', 'consumer', 'environmental', 'infrastructure', 'manufacturer', 'recycle', 'awareness', 'recovery', 'disposal', 'landfill', 'stewardship', 'stakeholder', 'raw', 'create', 'reduce', 'appropriate', 'Regulations', 'develop', 'improve', 'ensure', 'encourage', 'activity', 'economically', 'challenge', 'valuable', 'opportunity', 'benefit', 'new', 'sustainable', 'essential', 'hazardous', 'Manufacturers', 'need', 'efficient', 'avoid', 'effective', 'profitable', 'cost', 'long', 'circular', 'loop', 'issue', 'party', 'still', 'burden', 'environmentally', 'target', 'EOL', 'effort', 'make', 'proper', 'developing', 'reducing', 'promote', 'necessary', 'social', 'future', 'fully', 'uncertainty', 'viability', 'financial', 'introducing', 'job', 'robust', 'rare', 'investm

# Load test set (can be replaced to other set/file)

In [2]:
import json

sens = []
opinion_answer = []
driver_answer = []
barrier_answer = []

with open('UNSW_answer_without_conclusion/sen_token.json') as file_obj:
    sen = json.load(file_obj)
    sens.extend(sen)
with open('UNSW_answer_without_conclusion/driver_token.json') as file_obj:
    driver = json.load(file_obj)
    driver_answer.extend(driver)
    opinion_answer.extend(driver)
with open('UNSW_answer_without_conclusion/barrier_token.json') as file_obj:
    barrier = json.load(file_obj)
    barrier_answer.extend(barrier)
    opinion_answer.extend(barrier)
with open('UNSW_answer_2/sen_token.json') as file_obj:
    sen = json.load(file_obj)
    sens.extend(sen)
with open('UNSW_answer_2/driver_token.json') as file_obj:
    driver = json.load(file_obj)
    driver_answer.extend(driver)
    opinion_answer.extend(driver)
with open('UNSW_answer_2/barrier_token.json') as file_obj:
    barrier = json.load(file_obj)
    barrier_answer.extend(barrier)
    opinion_answer.extend(barrier)
    
print(str(len(sens)) + ' sentences in total')
print(str(len(opinion_answer)) + ' are opinion')
print(len(opinion_answer)/len(sens))

695 sentences in total
153 are opinion
0.22014388489208633


In [27]:
find_opinion = find_opinion_by_lexicon(sens, 'final_200.json', 0.24)
print(len(find_opinion))

166


In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metric(find, answer, sents):
    y_actual = []
    y_predicted = []
    for s in sents:
        if s in answer:
            y_actual.append(1)
        else:
            y_actual.append(0)
        if s in find:
            y_predicted.append(1)
        else:
            y_predicted.append(0)

    Accuracy = accuracy_score(y_actual, y_predicted)
    Precision = precision_score(y_actual, y_predicted)
    Recall = recall_score(y_actual, y_predicted)
    F1_Score = f1_score(y_actual, y_predicted)

    return Precision, Recall, F1_Score, Accuracy

def print_detail(find, answer, sents, name):
    print(len(sents), 'samples in total')
    print(len(answer), name+'s in sample')
    print(len(find), name+'s are found')
    count = 0
    for f in find:
        if f in answer:
            count += 1
    print(count, name+'s found are right')

In [29]:
print('Precision, Recall, F1_Score, Accuracy')
print(calculate_metric(find_opinion, opinion_answer, sens))
print_detail(find_opinion, opinion_answer, sens, 'opinion')

Precision, Recall, F1_Score, Accuracy
(0.35542168674698793, 0.38562091503267976, 0.36990595611285265, 0.7107913669064748)
695 samples in total
153 opinions in sample
166 opinions are found
59 opinions found are right


## driver & barrier classification

In [13]:
import import_ipynb
from utils import construct_dict
import os

if 'opinion_dic.json' in os.listdir():
    with open('opinion_dic.json') as file_obj:
        opinion_dic = json.load(file_obj)
else:
    opinion_dic = construct_dict(opinions)
    
driver_dic = construct_dict(drivers)
barrier_dic = construct_dict(barriers)

In [14]:
import numpy as np
from sklearn import preprocessing

driver_words = []
driver_freq = []
opinion_freq = []
for dd in driver_dic.keys():
    if dd in opinion_dic.keys():
        driver_words.append(dd)
        driver_freq.append(driver_dic[dd])
        opinion_freq.append(opinion_dic[dd])
print(len(opinion_dic.keys()), 'words in opinion_dic')
print(len(driver_dic.keys()), 'words in driver_dic')

# Sort(Normalization(Opinion_freq) - Normalization(lexicon_freq)) to construct dictionary

driver_freq = np.array(driver_freq).reshape(1, -1)
normalizer_driver = preprocessing.Normalizer().fit_transform(driver_freq)
normalizer_driver = list(normalizer_driver[0])

opinion_freq = np.array(opinion_freq).reshape(1, -1)
normalizer_opinion = preprocessing.Normalizer().fit_transform(opinion_freq)
normalizer_opinion = list(normalizer_opinion[0])

more_value = {}
for i, w in enumerate(driver_words):
    more_value[w] = normalizer_driver[i]-normalizer_opinion[i]
pri_words = sorted(more_value.items(), key=lambda item:item[1], reverse=True)
pri_list_whole = []
for p_set in pri_words:
    if opinion_dic[p_set[0]]>1:
        # print(i, p_set, opinion_dic[p_set[0]])
        pri_list_whole.append(p_set[0])
        
# The amount can be adjusted, here choose 100  
driver_pri = pri_list_whole[:100]
print(driver_pri)

3205 words in opinion_dic
2704 words in driver_dic
['management', 'environmental', 'industry', 'responsibility', 'also', 'reduce', 'resource', 'need', 'order', 'solar', 'impact', 'energy', 'benefit', 'government', 'producer', 'would', 'reduction', 'necessary', 'treatment', 'ensure', 'future', 'market', 'sustainable', 'approach', 'encourage', 'important', 'effective', 'opportunity', 'circular', 'system', 'support', 'policy', 'provide', 'improve', 'essential', 'waste', 'efficient', 'model', 'help', 'raw', 'supply', 'strategy', 'needed', 'manufacturer', 'new', 'economy', 'avoid', 'developing', 'sector', 'Therefore', 'required', 'sustainability', 'must', 'collection', 'business', 'stakeholder', 'manufacturing', 'appropriate', 'environment', 'take-back', 'consider', 'chain', 'job', 'solution', 'based', 'demand', 'al.', 'potential', 'responsible', 'best', 'European', 'mean', 'aim', 'price', 'party', 'positive', 'Si', 'growth', 'reduced', 'Directive', 'plan', 'improved', 'network', 'decrease'

In [15]:
import numpy as np
from sklearn import preprocessing

barrier_words = []
barrier_freq = []
opinion_freq = []
for dd in barrier_dic.keys():
    if dd in opinion_dic.keys():
        barrier_words.append(dd)
        barrier_freq.append(barrier_dic[dd])
        opinion_freq.append(opinion_dic[dd])
print(len(opinion_dic.keys()), 'words in opinion_dic')
print(len(barrier_dic.keys()), 'words in barrier_dic')

# Sort(Normalization(Opinion_freq) - Normalization(lexicon_freq)) to construct dictionary

barrier_freq = np.array(barrier_freq).reshape(1, -1)
normalizer_barrier = preprocessing.Normalizer().fit_transform(barrier_freq)
normalizer_barrier = list(normalizer_barrier[0])

opinion_freq = np.array(opinion_freq).reshape(1, -1)
normalizer_opinion = preprocessing.Normalizer().fit_transform(opinion_freq)
normalizer_opinion = list(normalizer_opinion[0])

more_value = {}
for i, w in enumerate(barrier_words):
    more_value[w] = normalizer_barrier[i]-normalizer_opinion[i]
pri_words = sorted(more_value.items(), key=lambda item:item[1], reverse=True)
pri_list_whole = []
for p_set in pri_words:
    if opinion_dic[p_set[0]]>1 and p_set[0] != '[' and p_set[0] != ']' and p_set[0] != '’':
        # print(i, p_set, opinion_dic[p_set[0]])
        pri_list_whole.append(p_set[0])
        
# The amount can be adjusted, here choose 100  
barrier_pri = pri_list_whole[:100]
print(barrier_pri)

3205 words in opinion_dic
1642 words in barrier_dic
['lack', 'current', 'low', 'still', 'challenge', 'incentive', 'due', 'metal', 'However', 'process', 'currently', 'high', 'present', 'many', 'yet', 'amount', 'barrier', 'available', 'limited', 'activity', 'value', 'higher', 'difficult', 'recycle', 'uncertainty', 'volume', 'long', 'issue', 'face', 'little', 'product', 'absence', 'consumer', 'major', 'given', 'landfill', 'type', 'handle', 'year', 'recyclers', 'facility', 'BESS', 'concentration', 'challenging', 'small', 'rate', 'According', 'site', 'transport', 'thermal', 'problem', '“', 'fact', 'may', 'however', 'recover', 'today', 'Regulations', 'generated', 'different', 'cost', 'poor', 'usually', 'country', 'economic', 'concern', 'i.e', 'plant', 'silicon-based', 'extremely', 'uncertain', 'confidence', 'focused', 'interesting', 'CUR', 'FIR', 'IBPR', 'LIR', 'LPR', 're-cycling', 'stream', 'even', 'considered', 'addressed', 'involved', 'lifetime', 'enough', 'reason', 'quantity', 'always', 

In [29]:
def db_(driver_p, barrier_p, opinions):
    find_driver = []
    for o in opinions:
        d_score = 0
        b_score = 0
        for word in o:
            if word in driver_p:
                d_score += 1
            if word in barrier_p:
                b_score += 1
        if d_score >= b_score:
            find_driver.append(o)
    return find_driver

In [18]:
sens = []
opinion_answer = []
driver_answer = []
barrier_answer = []

with open('UNSW_answer_without_conclusion/sen_token.json') as file_obj:
    sen = json.load(file_obj)
    sens.extend(sen)
with open('UNSW_answer_without_conclusion/driver_token.json') as file_obj:
    driver = json.load(file_obj)
    driver_answer.extend(driver)
    opinion_answer.extend(driver)
with open('UNSW_answer_without_conclusion/barrier_token.json') as file_obj:
    barrier = json.load(file_obj)
    barrier_answer.extend(barrier)
    opinion_answer.extend(barrier)
with open('UNSW_answer_2/sen_token.json') as file_obj:
    sen = json.load(file_obj)
    sens.extend(sen)
with open('UNSW_answer_2/driver_token.json') as file_obj:
    driver = json.load(file_obj)
    driver_answer.extend(driver)
    opinion_answer.extend(driver)
with open('UNSW_answer_2/barrier_token.json') as file_obj:
    barrier = json.load(file_obj)
    barrier_answer.extend(barrier)
    opinion_answer.extend(barrier)
    
print(str(len(sens)) + ' sentences in total')
print(str(len(opinion_answer)) + ' are opinion')
print(str(len(driver_answer)) + ' are driver')
print(len(opinion_answer)/len(sens))

695 sentences in total
153 are opinion
117 are driver
0.22014388489208633


In [40]:
find_driver = db_(driver_pri, barrier_pri, opinion_answer)
find_barrier = []
print(len(find_driver))
for o in opinion_answer:
    if o not in find_driver:
        find_barrier.append(o)

88


In [32]:
print('Precision, Recall, F1_Score, Accuracy')
print(calculate_metric(find_driver, driver_answer, opinion_answer))
print_detail(find_driver, driver_answer, opinion_answer, 'driver')

Precision, Recall, F1_Score, Accuracy
(0.8977272727272727, 0.6752136752136753, 0.7707317073170732, 0.6928104575163399)
153 samples in total
117 drivers in sample
88 drivers are found
79 drivers found are right


In [41]:
print('Precision, Recall, F1_Score, Accuracy')
print(calculate_metric(find_barrier, barrier_answer, opinion_answer))
print_detail(find_barrier, barrier_answer, opinion_answer, 'driver')

Precision, Recall, F1_Score, Accuracy
(0.4153846153846154, 0.75, 0.5346534653465346, 0.6928104575163399)
153 samples in total
36 drivers in sample
65 drivers are found
27 drivers found are right
